<a href="https://colab.research.google.com/github/fatimapinheir/EDA/blob/main/Tabela_de_Dispers%C3%A3o_EDA_UFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tabela de Dispersão - EDA - UFC 

Imprementar uma tabela de dispersão usando Simple Tabulation Hashing e endereçamento aberto com tentativa linear junto com table doubling e halving.

Devem ser implementadas as seguintes operações:

Incluir(inteiro x): inclui o inteiro de 64 bits x (manter mais de uma cópia caso seja repetido)
Remover(Inteiro x): remover uma ocorrência do inteiro x (encontrar e remover a primeira ocorrência e não fazer nada caso tal ocorrência não esteja na tabela)
Buscar(Inteiro x): acha a posição na tabela de um elemento que contém o inteiro x ou indica que tal elemento não está na tabela

ENTRADA:
Vocês precisam ler um arquivo de texto que será dado como entrada para realizar um conjunto de operações.
Cada linha deste arquivo pode conter as seguintes infermações

INC:x (incluir o inteiro x)
REM:x (remover o inteiro x)
BUS:x (busca pelo inteiro x)

Exemplo de um arquivo de entrada:

INC:10
INC:20
REM:10
INC:15
INC:15
REM:17
BUS:42
INC:42
INC:43


SAÍDA:
Você deve gerar um arquivo de texto como saída 
As linhas do arquivo de entrada serão copiadas no arquivo de saída separados por duas linhas.
Na linha imediatamente seguinte à cópia da linha de entrada, você deve indicar o resultado da operação no seguinte formato:

INC:x (incluir o inteiro x)
(valor de h(x) - função de dispersão sobre x) (posição onde o elemento entrou na tabela)

REM:x (remover o inteiro x)
(valor de h(x) - função de dispersão sobre x) (posição onde o elemento foi removido da tabela ou -1 se nenhum elemento foi removido)

BUS:x (busca pelo inteiro x)
(valor de h(x) - função de dispersão sobre x) (posição onde uma cópia do elemento está na tabela ou -1 se ele não está na tabela)

A primeira linha do seu arquivo de saída deve conter o tamanho inicial da tabela. Além disto, sempre que ocorrer um table doubling ou halving ou reconstrução da tabela por muitos elementos removidos, você deve indicar cada um no seguinte formato:
Na primeira linha, indicar o tamanho inicial da tabela
TAM:(tamanho inicial da tabela)

Adicionar esta linha logo após a inclusão de um elemento que dobra o tamanho da tabela.
DOBRAR TAM:(novo tamanho da tabela)

Adicionar esta linha logo após a remoção de um elemento que divide o tamanho da tabela.
METADE TAM:(novo tamanho da tabela)

Adicionar esta linha logo após a remoção de um elemento que limpa códigos de remoção
LIMPAR - indica a limpeza de códigos de remoção

Exemplo de saída:

TAM:4

INC:10
1 1

INC:20
3 3

REM:10
1 1

INC:15
0 0

INC:15
0 1

REM:17
2 -1

BUS:42
3 -1

INC:42
7 7

DOBRAR TAM:8

INC:43
2 3

OBSERVAÇÃO:
Lembrar que na tentativa linear, você deve manter a tabela com tamanho pelo menos (1+epsilon)n, para algum epsilon escolhido por vocês.

In [1]:
import sys
import math
import random

Imprementar uma tabela de dispersão usando Simple Tabulation Hashing e endereçamento aberto com tentativa linear junto com table doubling e halving.



In [2]:
class TableElement:
    """Doc da classe"""
    def __init__(self, key, value):
        self._key = key
        self._value = value

    def get_key(self):
        return self._key
    
    def get_value(self):
        return self._value

Fale um pouco da Hash, use figuras se necessário

In [3]:
class Hash:
    """ Class doc """
    def __init__(self, _hash = None):
        self.table = [None]
        self.num_elements = 0 
        self.num_removed = 0 
        self.hash_keys = []

        self.create_hash_key(_hash)

    def create_hash_key(self, _hash):
        """ Doc """
        table_size = 2**8
        if _hash is None:
            for i in range(0, 8):
                table = []

                for j in range(table_size):
                    table.append(random.randint(0, 2**64))

                self.hash_keys.append(table)
        else:
            if len(_hash) != 8:
                raise Exception("Invalid Table")
            
            self.hash_keys = _hash


    def create_binary(self, chave):
        """ Doc """
        b = [int(i) for i in list('{0:0b}'.format(chave))]

        if len(b) < 64:
            b = [0] * (64 - len(b)) + b
        else:
            raise Exception("Invalid key")

        b = [b[i:i + 8] for i in range(0, len(b), 8)]
        return b

    def create_int(self, array_binario):
        """ Doc """
        return sum(b<<index for index, b in enumerate(array_binario[::-1]))

    def xor(self, x, y):
        return x ^ y

    def get_hash(self, chave):
        """ Doc """
        t = None

        bin_array = self.create_binary(chave)

        for index, element in enumerate(bin_array):
            if index > 0:
                t = self.xor(t, self.__tabela(index, self.create_int(element)))
            elif index == 0:
                t = self.create_int(element)

        return t

    def __tabela(self, index, chave):   
        return self.hash_keys[index][chave]

    def resizing(self, tamanho):
        """ Doc """
        old_table = self.table

        self.num_elements = 0
        self.num_removed = 0
        self.table = [None] * tamanho

        for element in old_table:
            if element != None and not math.isnan(element.get_key()):
                self.incluir(element)

    def check_doubling(self, m):
        """ Doc """
        if m < (2)*self.num_elements:
            self.resizing(len(self.table) * 2)
            m = len(self.table)
            return True, m
        else:
            return False, m

    def check_halving(self, m):
        """ Doc """
        if self.num_elements < m/4:
            self.resizing(math.ceil(len(self.table) / 2))
            m = len(self.table)
            return True, m
        else:
            return False, m
    
    def check_cleaning(self, m):
        """ Doc """
        print('Cleaning =' + str(self.num_removed/m > 0.25) + ' | num_removed= ' + str(self.num_removed) + ' | m=' + str(m))
        if self.num_removed/m >= 0.25:
            self.resizing(len(self.table))
            m = len(self.table)
            return True
        else:
            return False


    def incluir(self, entrada):
        """ Doc """
        self.num_elements = self.num_elements + 1

        h = self.get_hash(entrada.get_key())
        m = len(self.table)
        t = h % m
        i = 1

        while self.table[t] != None and not math.isnan(self.table[t].get_key()):
            t = (h + i) % m
            i = i + 1

        self.table[t] = entrada

        doubling, m = self.check_doubling(m)

        return [h, t, doubling, m]


    def remover(self, chave):
        """ Doc """
        self.num_elements = self.num_elements - 1

        h = self.get_hash(chave)
        m = len(self.table)
        t = h % m
        i = 1

        while self.table[t] != None and (self.table[t].get_key() != chave or math.isnan(self.table[t].get_key())):
            t = (h + i) % m
            i = i + 1

        if self.table[t] != None and self.table[t].get_key() == chave:
            self.table[t] = TableElement(float("NaN"),float("NaN"))
            self.num_removed = self.num_removed + 1

            halving, m = self.check_halving(m)
            cleaning = self.check_cleaning(m)

            return [h, t, cleaning, halving, m]
    
        else:
            return [h, -1, False, False, m]


    def buscar(self, chave):
        """ Doc """
        if chave == None:
            return None

        h = self.get_hash(chave)
        m = len(self.table)

        t = h % m
        i = 1

        while self.table[t] != None and self.table[t].get_key() != chave:
            t = (h + i) % m
            if self.table[t] != None:
                i = i + 1

        if self.table[t] != None and self.table[t].get_key() == chave:
            return [h, self.table[t].get_value()]
        else:
            return [h, -1]

Fale um pouco dessa função 

In [4]:
def write_output(output, operation, value, h, t):
    """ Doc """
    output.write(operation + ":" + value + "\n\n")
    output.write(str(h) + " " + str(t) + "\n\n")

Fale um pouco dessa função

In [5]:
def check_operacao(comando, output):
    """ Doc """
    operacao, valor = comando

    if operacao == 'INC':
        h, t, doubling, m = tabela.incluir(TableElement(int(valor), int(valor)))
        write_output(output, operacao, valor, h, t)

        if doubling:
                output.write("DOBRAR TAM:" + str(m) + "\n\n")

    elif operacao == 'REM':
        h, t, cleaning, halving, m = tabela.remover(int(valor)) 
        write_output(output, operacao, valor, h, t)

        if halving:
                output.write("METADE TAM:" + str(m) + "\n\n")
        if cleaning:
            output.write("LIMPAR\n\n")

    elif operacao == 'BUS':
        h, t = tabela.buscar(int(valor))
        write_output(output, operacao, valor, h, t)

# E Aqui tudo acontece

...

In [6]:
#lendo input
tabela = Hash()

diretorio = "/content/"

output = open(diretorio + 'output.txt', 'w')
output.write("TAM:" + str(len(tabela.table)) + "\n\n")

input_ = open(diretorio + "input.txt", 'r')

for line in input_.readlines():
    command = line.strip().split(':')
    check_operacao(command, output)

Cleaning =False | num_removed= 1 | m=32
Cleaning =False | num_removed= 2 | m=32
Cleaning =False | num_removed= 3 | m=32
Cleaning =False | num_removed= 4 | m=32
Cleaning =False | num_removed= 5 | m=32
Cleaning =False | num_removed= 6 | m=32
Cleaning =False | num_removed= 7 | m=32
Cleaning =False | num_removed= 8 | m=32
Cleaning =False | num_removed= 0 | m=16
Cleaning =False | num_removed= 1 | m=16
Cleaning =False | num_removed= 2 | m=16
Cleaning =False | num_removed= 3 | m=16
Cleaning =False | num_removed= 0 | m=8
Cleaning =False | num_removed= 1 | m=8
Cleaning =False | num_removed= 0 | m=4
Cleaning =False | num_removed= 0 | m=2
